## Transfer Learning using MNIST data
To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

def view_layer(model):
    print('='*100)
    trainable_params = non_trainable_params = 0
    for i, layer in enumerate(model.layers, start=1):
        if layer.weights:
            n_params = K.count_params(layer.weights[0])+K.count_params(layer.weights[1])
            if layer.trainable:
                trainable_params += n_params
            else:
                non_trainable_params += n_params
            print('Layer {:02d}: {:15} - Trainable: {:5} - #Params: {:9,d} - Output Shape: {}'.format( \
                i, layer.name[:15], str(layer.trainable), n_params, layer.output_shape))
        else:
            print('Layer {:02d}: {:15} - Trainable: {:5} - {} - Output Shape: {}'.format( \
                i, layer.name[:15], str(layer.trainable), 18*' ', layer.output_shape))  
    print('='*100)
    print('Total params: {:,d}'.format(trainable_params + non_trainable_params))
    print('Trainable params: {:,d}'.format(trainable_params))
    print('Non-trainable params: {:,d}'.format(non_trainable_params))

In [2]:
#used to help some of the timing functions
now = datetime.datetime.now

In [3]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 10

In [4]:
# set some more parameters
# MNIST 28x28

img_rows, img_cols = 28, 28

In [5]:
## This just handles some variability in how the input data is loaded
# if backend requied inoput shape channels first

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [6]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training


# must convert dat ato float32 !! It's faster than float64
# if no problem in precition of 32

def train_model(model, train, test, num_classes, epochs):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    # retype to float32
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    # normalization
    x_train /= 255
    x_test /= 255
    
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [7]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

11490434/11490434 [==============================] - 2s 0us/step


separate feature layer

In [8]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(16, kernel_size=(3, 3),
           padding='valid',
           input_shape=input_shape, activation='relu'),
    Conv2D(16, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten()
]

separate classifier layer

In [9]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
]

In [10]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

2024-01-10 13:36:59.384119: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-01-10 13:36:59.384138: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-01-10 13:36:59.384143: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-01-10 13:36:59.384189: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-10 13:36:59.384216: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
# Let's take a look
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 16)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 16)        0         
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 128)               295040    
                                                        

In [12]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes, epochs)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/10


2024-01-10 13:37:00.080231: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


230/230 [==============================] - 6s 15ms/step - loss: 1.8716 - accuracy: 0.3577 - val_loss: 0.7897 - val_accuracy: 0.6785
Epoch 2/10
230/230 [==============================] - 3s 12ms/step - loss: 1.7843 - accuracy: 0.3323 - val_loss: 1.6963 - val_accuracy: 0.2076
Epoch 3/10
230/230 [==============================] - 3s 13ms/step - loss: 1.7141 - accuracy: 0.2038 - val_loss: 1.6150 - val_accuracy: 0.1971
Epoch 4/10
230/230 [==============================] - 3s 12ms/step - loss: 1.6811 - accuracy: 0.2034 - val_loss: 1.6602 - val_accuracy: 0.1835
Epoch 5/10
230/230 [==============================] - 3s 11ms/step - loss: 1.6774 - accuracy: 0.2033 - val_loss: 1.6592 - val_accuracy: 0.1835
Epoch 6/10
230/230 [==============================] - 3s 12ms/step - loss: 1.6675 - accuracy: 0.2061 - val_loss: 1.6756 - val_accuracy: 0.1835
Epoch 7/10
230/230 [==============================] - 3s 11ms/step - loss: 1.6740 - accuracy: 0.2027 - val_loss: 1.6194 - val_accuracy: 0.2004
Epoch 8/10

### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

### Looking Inside Model

## [Code for Viewing Layers](https://docs.google.com/document/d/1iACjBPWTWvXTSecBNHx1xEtODcHSpnU6HFY82b4b6Ys)

#### Copy and Paste the code here!

### Freeze CNN Layers (Low-level or Early Layers)

In [13]:
# model.layers[0].trainable
view_layer(model)

Layer 01: conv2d          - Trainable: True  - #Params:       160 - Output Shape: (None, 26, 26, 16)
Layer 02: conv2d_1        - Trainable: True  - #Params:     2,320 - Output Shape: (None, 24, 24, 16)
Layer 03: max_pooling2d   - Trainable: True  -                    - Output Shape: (None, 12, 12, 16)
Layer 04: dropout         - Trainable: True  -                    - Output Shape: (None, 12, 12, 16)
Layer 05: flatten         - Trainable: True  -                    - Output Shape: (None, 2304)
Layer 06: dense           - Trainable: True  - #Params:   295,040 - Output Shape: (None, 128)
Layer 07: dropout_1       - Trainable: True  -                    - Output Shape: (None, 128)
Layer 08: dense_1         - Trainable: True  - #Params:       645 - Output Shape: (None, 5)
Total params: 298,165
Trainable params: 298,165
Non-trainable params: 0


In [14]:
# Freeze only the 
for l in feature_layers:
    l.trainable = False


In [15]:
view_layer(model)

Layer 01: conv2d          - Trainable: False - #Params:       160 - Output Shape: (None, 26, 26, 16)
Layer 02: conv2d_1        - Trainable: False - #Params:     2,320 - Output Shape: (None, 24, 24, 16)
Layer 03: max_pooling2d   - Trainable: False -                    - Output Shape: (None, 12, 12, 16)
Layer 04: dropout         - Trainable: False -                    - Output Shape: (None, 12, 12, 16)
Layer 05: flatten         - Trainable: False -                    - Output Shape: (None, 2304)
Layer 06: dense           - Trainable: True  - #Params:   295,040 - Output Shape: (None, 128)
Layer 07: dropout_1       - Trainable: True  -                    - Output Shape: (None, 128)
Layer 08: dense_1         - Trainable: True  - #Params:       645 - Output Shape: (None, 5)
Total params: 298,165
Trainable params: 295,685
Non-trainable params: 2,480


Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 16)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 16)        0         
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 128)               295040    
                                                        

In [17]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes, epochs//4)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/2
240/240 [==============================] - 3s 12ms/step - loss: 1.6669 - accuracy: 0.2039 - val_loss: 3.1623 - val_accuracy: 0.1911
Epoch 2/2
240/240 [==============================] - 2s 10ms/step - loss: 1.6722 - accuracy: 0.2025 - val_loss: 2.8457 - val_accuracy: 0.1911
Training time: 0:00:05.512679
Test score: 2.8456904888153076
Test accuracy: 0.1910877674818039


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 

## Exercise
### To do:
- Now write code to reverse this training process.  That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.